In [1]:
import pandas as pd
import pysal as ps
import folium
import os
os.chdir("/media/sf_Shared_Linux/Project/")
from dataIO import dbf2df

In [2]:
geo_path=r'Shapefiles/LynnHaven_simplified3.json'
data_path=r'Data/S1_100yr_sum.dbf'
df = dbf2df(data_path)
df['BldgLoss'] = df['BldgLoss']/1000
df['ContLoss'] = df['ContLoss']/1000

In [3]:
import json
with open(geo_path) as data_file:    
    data = json.load(data_file)
    keys = []
    for i in range(len(data['features'])):
        keys.append(data['features'][i]['properties']['CensusBloc'])
data_file.close()

In [4]:
columns = ['BldCount','BldgLoss','ContLoss']
df_ = pd.DataFrame(index=keys, columns=columns)
df_ = df_.fillna(-1) # with 0s rather than NaNs
df.set_index('CensusBlock', inplace=True)
df_.update(df)
df_.index.name='CensusBlock'
df_.reset_index(inplace=True)

In [5]:
cartodb = folium.Map(location=[36.883,-76.092],
                    tiles = 'CartoDB positron',
                    zoom_start = 12)
#stamen.add_children(LatLngPopup)
cartodb.choropleth(geo_path=geo_path,
                  data = df_,
                  columns=['CensusBlock','BldgLoss','ContLoss'],
                  key_on='feature.properties.CensusBloc',
                  fill_color='PuBuGn',
                  fill_opacity=0.7,
                  line_opacity=0.2,
                  legend_name='Building Losses ($K)',
                  threshold_scale=[0,200,500,1000,2000,5000])
#folium.utilities.split_six
cartodb.save(outfile='cartodb_positron_S1_100yr.html')

In [ ]:
help(folium.Map)